<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module2/M2_2/6_TheSimplexMethod_Minimization3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Simplex Method: Minimization 3D
**OPIM 5604: Business Decision Modeling - University of Connecticut**

Material from: "Elementary Linear Algebra" - 8th Edition (Ron Larson) - Chapter 9.

---------------------------------------------------------------------------
**Objectives:**
* Determine the dual of a linear programming problem that  minimizes an objective function.  
* Use the simplex method to solve a linear programming problem that minimizes an objective function

In [ ]:
# import modules
# Matrix makes a sympy Matrix, Rational is FRACTION, pprint makes it pretty print, nsimplify converts decimals to fractions (rational numbers)
from sympy import Matrix, Rational, pprint, nsimplify 

# standard modules
import numpy as np
import pandas as pd

In [ ]:
# this function is useful for coloring a single cell
# you'll see how to use it later

# Custom function to color the desired cell
def styling_specific_cell(x,row_idx,col_idx):
    color = 'background-color: yellow; color: red'
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] = color
    return df_styler

**Background:** You previously applied the simplex method to linear programming problems in **standard form** where the objective function was to be **maximized** (all constraints had $\leq$ symbols). In this section, you will extend this procedure to linear programming problems in which the objective function is to be **minimized** (requires all constraints to have $\geq$ symbols). 

Specifically, a minimization problem is defined in standard form when the objective function:

$w = c_1x_1 + c_2x_2 + ... c_nx_n$

is to be minimized, subject to the constraints

$a_m$$_1x_1 + $ $a_m$$_2x_2 + $ $...+ a_m$$_nx_n \geq b_m$

where $x_i \geq 0$ and $b_i \geq 0$.

The basic procedure used to solve such a problem is to convert it to a **maximization problem** in standard form, then apply the Simplex method. That's we are starting off by using $w$ instead of $z$ in the objective function as a helpful reminder. At the end, you just need to interpret/read off the final output (tableau) a little differently.

Let's consider a simple problem in 3D.


#Problem Description
Find the minimum value of
$w = 2x_1 + 10x_2 + 8x_3$

subject to:
* $x_1 + x_2 + x_3\geq 6$
* $x_2 + 2x_3 \geq 8$
* $-x_1 + 2x_2 + 2x_3 \geq 4$
* $x_1,x_2 , x_3 \geq 0$

First of all - **notice that $z$ is nowhere to be found** - as a naming convention, we will use $w$ for minimization problems to help keep us organized.

To solve this problem using simplex, you must convert it to a maximization problem. The first step is to form the augmented matrix for this system of inequalities. To this augmented matrix, add a last row that represents the coefficients of the objective function and don't rearrange terms.

**Note that we have not rearranged terms for the objective function.** The values of each of the decision variables are still the same.

In our example, $A$ will always be the matrix, and `tmp` will always be a temporary copy of $A$ that we will color code. See helper function at the end to color a pandas dataframe, rename rows and columns etc.



## Set up initial matrix ($A$)

In [ ]:
# form the matrix
A = Matrix([[1, 1, 1, 6],
            [0, 1, 2, 8],
            [-1,2,2,4],
            [2,10,8,0]])

# show the matrix
pprint(A)

⎡1   1   1  6⎤
⎢            ⎥
⎢0   1   2  8⎥
⎢            ⎥
⎢-1  2   2  4⎥
⎢            ⎥
⎣2   10  8  0⎦


In [ ]:
# if you want to make it pretty, you can use pandas
# example
# link: https://stackoverflow.com/questions/40335140/how-to-highlight-both-a-row-and-a-column-at-once-in-pandas

# make sure you convert A to a numpy array for easy viewing - and make it a float 
# so that stuff isn't stored as a string
tmp = pd.DataFrame(np.array(A).astype(float))

tmp.columns = ['x1', 'x2', 'x3','b']
tmp.index=['R0', 'R1', 'R2', 'R3']

# w is in the bottom right cell

# style - this is for an individual cell
idx_c = 3   # Column index of cell to color 
idx_r = 3   # Row index of cell to color

# see the columns 's1' and 's2' called out in the code below?
tmp.style\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,x1,x2,x3,b
R0,1.000000,1.000000,1.000000,6.000000
R1,0.000000,1.000000,2.000000,8.000000
R2,-1.000000,2.000000,2.000000,4.000000
R3,2.000000,10.000000,8.000000,0.000000


## Transpose($A$)

Now, take the transpose of this matrix. Notice how each column becomes a row - the first row has become the first column - stare at this for a moment if it is new to you.

Also note that we have not added any slack variables yet! We aren't there yet.

In [ ]:
# transpose
# we will keep overwriting A
A = A.T
pprint(A)

⎡1  0  -1  2 ⎤
⎢            ⎥
⎢1  1  2   10⎥
⎢            ⎥
⎢1  2  2   8 ⎥
⎢            ⎥
⎣6  8  4   0 ⎦


When we take the transpose, we change our variables from $x$ to $y$ to help keep us organized.

In [ ]:
# you can make it pretty with pandas
# we have not added slack variables yet (coming soon)
tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['y1', 'y2', 'y3', 'b']
tmp.index=['R0', 'R1', 'R2','R3']
tmp

,y1,y2,y3,b
R0,1.0,0.0,-1.0,2.0
R1,1.0,1.0,2.0,10.0
R2,1.0,2.0,2.0,8.0
R3,6.0,8.0,4.0,0.0


Great! And now we will interpret this transposed matrix as a maximization problem. Will add our slack variables in a moment.

# Dual Maximization Problem
In order to interpret the transposed matrix as a maximization problem, we introduce new variables $y_1, y_2, y_3$. This corresponding maximization problem is called the dual of the original problem.

So, now we need to find the maximum value of:

$z = 6y_1 + 8y_2 + 4y_3$

subject to:
* $y_1      - y_3 + s_1 \leq 2$
* $y_1 + y_2 + 2y_3 + s_2 \leq 10$
* $y_1 + 2y_2 + 2y_3 + s_3 \leq 8$
* $y_1,y_2,y_3 \geq 0$

Do you see how all of the $\geq$ constraints were switched to $\leq$ (does not include nonnegativity, which is always true)? 

Notice our naming conventions here - $w$ has been replaced by $z$, and $x$ has been replaced by $y$. This naming convention will help keep you organized!

The solution of the original minimization problem can be found by applying the simplex method to the new dual problem. At the end, you'll just need to interpret (read off) the final tableau a little differently.

Let's remake A but now we will include our slack variables $s_1$ , $s_2$ and $s_3$.

## Standard Form

In [ ]:
# since this is a maximization problem
# we need to remake our initial tableau and add slack variables
# and we are overwriting A


# names are y1, y2, y3, s1, s2,s3,  b
A = Matrix([[1,0,-1,1,0,0,2],
            [1,1,2,0,1,0,10],
            [1,2,2,0,0,1,8],
            [-6,-8,-4,0,0,0,0]])

pprint(A)

⎡1   0   -1  1  0  0  2 ⎤
⎢                       ⎥
⎢1   1   2   0  1  0  10⎥
⎢                       ⎥
⎢1   2   2   0  0  1  8 ⎥
⎢                       ⎥
⎣-6  -8  -4  0  0  0  0 ⎦


In [ ]:
# make it pretty
tmp = pd.DataFrame(np.array(A).astype(float)) # you only need this in the first example
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2','s3', 'b']
tmp.index=['R0', 'R1', 'R2','R3']
tmp

# style - this is for an individual cell
idx_r = 3   # Row index of cell to color
idx_c = 6   # Column index of cell to color 

# see the columns 's1' and 's2', 's3' called out in the code below?
# see how I am apply the highlighting of columns twice?
# the slash \ at the end simply continues the code
tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 's1' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 's2' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 's3' else '' for i in x])\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)


,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,1.000000,1.000000,2.000000,0.000000,1.000000,0.000000,10.000000
R2,1.000000,2.000000,2.000000,0.000000,0.000000,1.000000,8.000000
R3,-6.000000,-8.000000,-4.000000,0.000000,0.000000,0.000000,0.000000


Do you see how we have incorporated our slack variabes $s_1$ ,$s_2$ and $s_3$ (blue- they correspond to values of 2, 10 and 8 - this is the 'do nothing' solution and we get a $z$ of 0)? And do you see how we have negative values in the bottom row? $z$ is in the bottom right (yellow/red cell). 

The **basic variables** are s1, s2 and s3 and this results in a $z$ of 0. We can do better than that!

Let's move onto pivoting...

# Reminder on Pivoting

Remmember - after you have set up the initial simplex tableau for a linear programming problem, the simplex method consists of checking for optimality and then, when the current solution is not optimal, improving the current solution. (An improved solution is one that has a larger z-value than the current solution.) 

To improve the current solution, bring a new basic variable into the solution, the entering variable. This implies that one of the current basic variables (the departing variable) must leave, otherwise you would have too many variables for a basic solution. Choose the entering and departing variables as listed below. 

1. The **entering variable** corresponds to the least (the most negative) entry in the bottom row of the tableau, excluding the “b-column.” 
2.  The **departing variable** corresponds to the least nonnegative ratio $b_i/$$a_i$$_j$ in the column determined by the entering variable, when $a_i$$_j$ $> 0$.
3.  The entry in the simplex tableau in the entering variable’s column and the departing variable’s row is the **pivot**. 

Finally, to form the improved solution, apply Gauss-Jordan elimination (https://online.stat.psu.edu/statprogram/reviews/matrix-algebra/gauss-jordan-elimination) to the column that contains the pivot, as illustrated in Example 1.
 

# Pivot #1


In [ ]:
# as a reminder, here is our matrix
tmp

,y1,y2,y3,s1,s2,s3,b
R0,1.0,0.0,-1.0,1.0,0.0,0.0,2.0
R1,1.0,1.0,2.0,0.0,1.0,0.0,10.0
R2,1.0,2.0,2.0,0.0,0.0,1.0,8.0
R3,-6.0,-8.0,-4.0,0.0,0.0,0.0,0.0


## Entering Variable
The entering variable is simply the most negative value in the bottom row, which is -8 ($y_2$).

## Departing Variable
Now, compute ratios $b_i/$$a_i$$_j$ to determine which variable is the **departing variable**.

$2/0 = \text{Inf}$ `R0`

$10/1 = 10$ `R1`

$8/2 = 4$ `R2`

Since $R2$ is the *smaller positive number*, it becomes the **departing variable**.

In [ ]:
#print(2/0) # R0 - invalid
print(10/1) # R1
print(8/2) # R2

10.0
4.0


## Turn Pivot Element Into a '1'.

Now that means the pivot element is '2' in the left corner, which we will need to turn into a 1. 

In [ ]:
# before
pprint(A)

⎡1   0   -1  1  0  0  2 ⎤
⎢                       ⎥
⎢1   1   2   0  1  0  10⎥
⎢                       ⎥
⎢1   2   2   0  0  1  8 ⎥
⎢                       ⎥
⎣-6  -8  -4  0  0  0  0 ⎦


In [ ]:
# before - make it pretty
tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2','s3', 'b']
tmp.index=['R0', 'R1', 'R2' , 'R3']
tmp

# highlight the pivot element
idx_r = 2
idx_c = 1

tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y2' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R2' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,1.000000,1.000000,2.000000,0.000000,1.000000,0.000000,10.000000
R2,1.000000,2.000000,2.000000,0.000000,0.000000,1.000000,8.000000
R3,-6.000000,-8.000000,-4.000000,0.000000,0.000000,0.000000,0.000000


Now turn that pivot element into a 1!

In [ ]:
# after
A[2,:] = Rational(1,2)*A[2,:]
pprint(A) # check your work

⎡ 1   0   -1  1  0   0   2 ⎤
⎢                          ⎥
⎢ 1   1   2   0  1   0   10⎥
⎢                          ⎥
⎢1/2  1   1   0  0  1/2  4 ⎥
⎢                          ⎥
⎣-6   -8  -4  0  0   0   0 ⎦


You don't need to always run this code, but you will want to when there are decimals in your matrix and it's unclear how to simplify them.

In [ ]:
# if it looks a little messy, try nsimplify
# make sure you wrap it in a Matrix()...
# otherwise you will get an 'immutable matrix' error
A = Matrix(nsimplify(A, rational=True))
pprint(A)

⎡ 1   0   -1  1  0   0   2 ⎤
⎢                          ⎥
⎢ 1   1   2   0  1   0   10⎥
⎢                          ⎥
⎢1/2  1   1   0  0  1/2  4 ⎥
⎢                          ⎥
⎣-6   -8  -4  0  0   0   0 ⎦


In [ ]:
# make it pretty
tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2','s3', 'b']
tmp.index=['R0', 'R1', 'R2' , 'R3']
tmp

# highlight the pivot element
idx_r = 2
idx_c = 1

tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y2' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R2' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,1.000000,1.000000,2.000000,0.000000,1.000000,0.000000,10.000000
R2,0.500000,1.000000,1.000000,0.000000,0.000000,0.500000,4.000000
R3,-6.000000,-8.000000,-4.000000,0.000000,0.000000,0.000000,0.000000


## Use GJ elimination to turn all other values above and below pivot element into 0s.

Great! Now we need to get rid of the $-8$ that is below the pivot element.

Leave the row of interest ($R0$ and $R1$) as-is, and then add multiples of $R2$ (the row with the pivot element.)

### R3

In [ ]:
# R3 = R3 + 8*R2
A[3,:] = A[3,:] + 8*A[2,:]

pprint(A)

⎡ 1   0  -1  1  0   0   2 ⎤
⎢                         ⎥
⎢ 1   1  2   0  1   0   10⎥
⎢                         ⎥
⎢1/2  1  1   0  0  1/2  4 ⎥
⎢                         ⎥
⎣-2   0  4   0  0   4   32⎦


### R1

In [ ]:
#R1 = R1 - R2
A[1,:] = A[1,:] - A[2,:]

pprint(A)

⎡ 1   0  -1  1  0   0    2 ⎤
⎢                          ⎥
⎢1/2  0  1   0  1  -1/2  6 ⎥
⎢                          ⎥
⎢1/2  1  1   0  0  1/2   4 ⎥
⎢                          ⎥
⎣-2   0  4   0  0   4    32⎦


In [ ]:
# make it pretty
tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2','s3', 'b']
tmp.index=['R0', 'R1', 'R2' , 'R3']
tmp

# highlight the pivot element
idx_r = 2
idx_c = 1

tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y2' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R2' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,0.500000,0.000000,1.000000,0.000000,1.000000,-0.500000,6.000000
R2,0.500000,1.000000,1.000000,0.000000,0.000000,0.500000,4.000000
R3,-2.000000,0.000000,4.000000,0.000000,0.000000,4.000000,32.000000


We have a '1' in the pivot element and '0's below thanks to Gauss-Jordan elimination. Great! 

That's all for the first pivot, check for negative values in the bottom row. If they exist (and they do...), you need to pivot again.

## Check your work - feasible solution?
Yes there is, but we will read it off in a minute.

### Any negative numbers in the bottom row?
Yes, there is still a $-2$ in the bottom row. We have to keep going.

# Pivot #2

## Entering Variable
OK! We checked and still see that there are negative elements in the bottom row. We see a $-2$ in the bottom row, so $y_1$ will become our **entering variable**.
 

In [ ]:
# make it pretty
tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2','s3', 'b']
tmp.index=['R0', 'R1', 'R2' , 'R3']
tmp


tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y1' else '' for i in x])

,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,0.500000,0.000000,1.000000,0.000000,1.000000,-0.500000,6.000000
R2,0.500000,1.000000,1.000000,0.000000,0.000000,0.500000,4.000000
R3,-2.000000,0.000000,4.000000,0.000000,0.000000,4.000000,32.000000


## Departing Variable

Our **departing variable** is going to be b/a...

In [ ]:
print(2/1) # R0 winner!
print(6/.5) # R1
print(4/.5) # R2


2.0
12.0
8.0


In [ ]:
# let's show the new entering and departing variables
# make it pretty


tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2','s3', 'b']
tmp.index=['R0', 'R1', 'R2' , 'R3']
tmp

# highlight the pivot element
idx_r = 0
idx_c = 0

tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y1' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R0' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)



,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,0.500000,0.000000,1.000000,0.000000,1.000000,-0.500000,6.000000
R2,0.500000,1.000000,1.000000,0.000000,0.000000,0.500000,4.000000
R3,-2.000000,0.000000,4.000000,0.000000,0.000000,4.000000,32.000000


## Turn the pivot element into a '1'

And it already is!

In [ ]:
pprint(A)

⎡ 1   0  -1  1  0   0    2 ⎤
⎢                          ⎥
⎢1/2  0  1   0  1  -1/2  6 ⎥
⎢                          ⎥
⎢1/2  1  1   0  0  1/2   4 ⎥
⎢                          ⎥
⎣-2   0  4   0  0   4    32⎦


## Use GJ elimination to turn all other values above and below the pivot element into 0s.

In [ ]:
# looks good! now we need to get rid of the
# 1/2 in R1
# 1/2 in R2
# -2 in R3

# remember, leave the row of interest as-is
# and add multiples of the row with the pivot element

# fix second row
A[1,:] = A[1,:] - Rational(1,2)*A[0,:]
# fix third row
A[2,:] = A[2,:] - Rational(1,2)*A[0,:]
# fix fourth row
A[3,:] = A[3,:] + 2*A[0,:]

# remember our titles
pprint(A) # WOOF! let's make it look nicer

⎡1  0  -1    1    0   0    2 ⎤
⎢                            ⎥
⎢0  0  3/2  -1/2  1  -1/2  5 ⎥
⎢                            ⎥
⎢0  1  3/2  -1/2  0  1/2   3 ⎥
⎢                            ⎥
⎣0  0   2    2    0   4    36⎦


## Check your work - feasible solution?
Yes, but wait a second for how to read it off!

### Any negative values in bottom row?
Let's take a look at what we did. Are there any negative values in the bottom row... nope! You are done.

In [ ]:
# let's show the new entering and departing variables
# make it pretty


tmp = pd.DataFrame(np.array(A).astype(float))
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2','s3', 'b']
tmp.index=['R0', 'R1', 'R2' , 'R3']
tmp

# highlight the pivot element
idx_r = 0
idx_c = 0

tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y1' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R0' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)


,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,0.000000,0.000000,1.500000,-0.500000,1.000000,-0.500000,5.000000
R2,0.000000,1.000000,1.500000,-0.500000,0.000000,0.500000,3.000000
R3,0.000000,0.000000,2.000000,2.000000,0.000000,4.000000,36.000000


# Reading the tableau
Do you see any negative pivot elements in the bottom? NOPE! So you are good to go. Time to read off the solution (be careful - it's different now!)

The solutional of the dual maximization problem is $z = 36$. So, the minmum value of w is 36, and this occurs when the slack variables, $x_1 = 2$, $x_2 = 0$, and $x_3 = 4$.

Now you read it by paying attention to your **slack variables!!**! We have to bring back our original objective function. 

$w = 2x_1 + 10x_2 + 8x_3$

$x_2 = 2$!

In [ ]:
# highlight the pivot element
# s1 (x1)
idx_r = 3
idx_c = 3

tmp.style\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,0.000000,0.000000,1.500000,-0.500000,1.000000,-0.500000,5.000000
R2,0.000000,1.000000,1.500000,-0.500000,0.000000,0.500000,3.000000
R3,0.000000,0.000000,2.000000,2.000000,0.000000,4.000000,36.000000


In [ ]:
# s2 (x2)
idx_r = 3
idx_c = 4

tmp.style\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,0.000000,0.000000,1.500000,-0.500000,1.000000,-0.500000,5.000000
R2,0.000000,1.000000,1.500000,-0.500000,0.000000,0.500000,3.000000
R3,0.000000,0.000000,2.000000,2.000000,0.000000,4.000000,36.000000


In [ ]:
# s3 (x3)
idx_r = 3
idx_c = 5

tmp.style\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)

,y1,y2,y3,s1,s2,s3,b
R0,1.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,2.000000
R1,0.000000,0.000000,1.500000,-0.500000,1.000000,-0.500000,5.000000
R2,0.000000,1.000000,1.500000,-0.500000,0.000000,0.500000,3.000000
R3,0.000000,0.000000,2.000000,2.000000,0.000000,4.000000,36.000000




The objective value $w$ of a minimization problem in standard form has a  minimum value if and only if the objective value $z$ of the dual maximization problem has a maximum value. Moreover, the minimum value of $w$ is equal to the maximum value of $z$.

Recall our original objective function:
$w = 2x_1 + 10x_2 + 8x_3$


# Appendix: Color coding a table
Use this script to update the colors in your tables. You can use this all over the place.

In [ ]:
# a random sample array

A = Matrix([[60,   12,   10,   1,  0,  0.12],
            [60,    6,   30,   0,  1,  0.15],                      
            [-300,  -36,  -90,  0,  0,   0]])

# make it pretty
tmp = pd.DataFrame(np.array(A).astype(float)) # you only need this in the first example
tmp.columns = ['y1', 'y2', 'y3', 's1', 's2', 'b']
tmp.index=['R0', 'R1', 'R2']
tmp

# add some color to highlight s1 and s2 and z
# Custom function to color the desired cell
def styling_specific_cell(x,row_idx,col_idx):
    color = 'background-color: yellow; color: red'
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] = color
    return df_styler

# highlight a single cell pivot element
idx_r = 0
idx_c = 0

# apply style to rows and columns
tmp.style\
.apply(lambda x: ['background: lightblue' if x.name == 'y1' else '' for i in x])\
.apply(lambda x: ['background: lightblue' if x.name == 'R0' else '' for i in x], axis=1)\
.apply(styling_specific_cell, row_idx = idx_r, col_idx = idx_c, axis = None)


,y1,y2,y3,s1,s2,b
R0,60.000000,12.000000,10.000000,1.000000,0.000000,0.120000
R1,60.000000,6.000000,30.000000,0.000000,1.000000,0.150000
R2,-300.000000,-36.000000,-90.000000,0.000000,0.000000,0.000000
